In [ ]:
import numpy as np

In [ ]:
# Datos
carga_inicial = 20
aumento_de_P = 0.25
carga_final = carga_inicial + 18 * aumento_de_P

d_AD = 1
d_AB = 2
d_BC = 2
d_AC = d_AB + d_BC

angulo_barra_CE = np.arctan(d_AD / d_BC)
angulo_barra_AE = np.arctan(d_AD / d_AB)

In [ ]:
# Definir el rango de los ángulos para el cambio de P
rango_de_angulos = np.arange(0,190,10)
rango_de_cargas = np.arange(carga_inicial, carga_final + aumento_de_P , aumento_de_P)
rango_de_cargas

In [ ]:
# Estática de la armadura
def calcular_reacciones(carga, angulo, d_AC): 
    angulo_rad = (angulo if angulo <= 90 else 100 - angulo) * np.pi / 180
    Ax = - Dx + carga * np.sin(angulo_rad)
    Ay = (-1 if angulo <= 90 else 1) * carga * np.cos(angulo_rad)
    Dx = (-1 if angulo <= 90 else 1) * d_AC * carga * np.cos(angulo_rad)
    return Ax, Ay, Dx

In [ ]:
# Análisis de nodos
def obtener_fuerzas_nodo_C(carga, angulo):
    angulo_rad = (angulo if angulo <= 90 else 100 - angulo) * np.pi / 180
    CE = (1 if angulo <= 90 else -1) * carga * np.cos(angulo_rad) / np.sin(angulo_barra_CE)
    BC = carga * np.sin(angulo_rad) - CE * np.cos(angulo_barra_CE)
    return CE, BC
    
def obtener_fuerzas_nodo_B(fuerza_BC):
    BA = fuerza_BC
    BE = 0
    return BA, BE

def obtener_fuerzas_nodo_E(fuerza_CE, angulo_barra_AE):
    EA = - fuerza_CE * np.sin(angulo_barra_CE) / np.sin(angulo_barra_AE)
    ED = - EA * np.cos(angulo_barra_AE) * fuerza_CE * np.cos(angulo_barra_CE)
    return EA, ED
    
def obtener_fuerzas_nodo_D(fuerza_Dx):
    ED = - fuerza_Dx
    DA = 0
    return ED , DA

def obtener_fuerzas_nodo_A(Ax, Ay, angulo_barra_AE):
    AE = Ay / np.sin(angulo_barra_AE)
    AB = - Ax - AE * np.cos(angulo_barra_AE)
    return AE, AB

In [ ]:
fuerza_maxima = np.array([0, 0, 0, 0, 0, 0, 0])
for angulo, carga in zip(rango_de_angulos, rango_de_cargas):
    Ax, Ay, Dx = calcular_reacciones(carga, angulo, d_AC)
    CE, BC = obtener_fuerzas_nodo_C(carga, angulo)
    BA, BE = obtener_fuerzas_nodo_B(fuerza_BC)
    EA, ED = obtener_fuerzas_nodo_E(fuerza_CE, angulo_barra_AE)
    ED , DA = obtener_fuerzas_nodo_D(fuerza_Dx)
    AE, AB = obtener_fuerzas_nodo_A(Ax, Ay, angulo_barra_AE)
    print(f"Reacciones angulo {angulo} grados:")
    print(Ax, Ay, Dx)
    print("Fuerzas internas:")    
    print(AB ,BC ,CD ,DE ,AE ,AD ,BD)
    print(f"{AE} == {EA}, {AB} == {BA}, {DE} == {ED}")
    print()
    fuerza_maxima[0] = BA if np.absolute(BA) > fuerza_maxima[0] else fuerza_maxima[0]
    fuerza_maxima[1] = BC if np.absolute(BC) > fuerza_maxima[1] else fuerza_maxima[1]
    fuerza_maxima[2] = CE if np.absolute(CE) > fuerza_maxima[2] else fuerza_maxima[2]
    fuerza_maxima[3] = ED if np.absolute(ED) > fuerza_maxima[3] else fuerza_maxima[3]
    fuerza_maxima[4] = EA if np.absolute(EA) > fuerza_maxima[4] else fuerza_maxima[4]
    fuerza_maxima[5] = DA if np.absolute(DA) > fuerza_maxima[5] else fuerza_maxima[5]
    fuerza_maxima[6] = BE if np.absolute(BE) > fuerza_maxima[6] else fuerza_maxima[6]
    
print(fuerza_maxima)

In [ ]:
# PARTE 2
# Mpa / N / mm^2
esfuerzo_admisible = 21
# mm^2
area = 1200

In [ ]:
# KN
fuerza_maxima = esfuerzo_admisible * area  / 1000
fuerza_maxima

In [ ]:
fuerzas_internas = np.array([0, 0, 0, 0, 0, 0, 0])
for angulo, carga in zip((rango_de_angulos, rango_de_cargas)
    Ax, Ay, Dx = calcular_reacciones(carga, angulo, d_AC)
    CE, BC = obtener_fuerzas_nodo_C(carga, angulo)
    BA, BE = obtener_fuerzas_nodo_B(fuerza_BC)
    EA, ED = obtener_fuerzas_nodo_E(fuerza_CE, angulo_barra_AE)
    ED , DA = obtener_fuerzas_nodo_D(fuerza_Dx)
    AE, AB = obtener_fuerzas_nodo_A(Ax, Ay, angulo_barra_AE)
    fuerzas_internas = np.array([("AB", AB), ("AE", AE), ("AD", AD), ("BC", BC), ("BE", BE), ("CE", CE), ("ED", ED)])
    elementos_fallan = np.where(np.absolute(np.array([x[1] for x in fuerzas_internas], dtype=np.float64)) > fuerza_maxima)
    if len(elementos_fallan) > 0:
       print(f"La primera falla se presenta en la carga de {carga} KN y en la dirección de {angulo} grados ")
       for i in elementos_fallan[0]:
            fuerza = fuerzas_internas[i][1].astype(float)
            causa_falla = "Compresión" if fuerza < 0 else "Tracción"
            print(f"El elemento {fuerzas_internas[i][0]} falla bajo la carga {np.absolute(fuerza)} KN a {causa_falla}")
        print() 

In [ ]:
# PARTE 3
# Código 2195566 L = 5
L = 5
aumento_distancia = float("0.0" + str(L))
A = 6
B = 6
AB = 50
AB = round(AB / 10) * 10
AB, aumento_distancia

In [ ]:
for angulo, carga in zip(rango_de_angulos, rango_de_cargas):
    d_AC += aumento_distancia
    d_AB += aumento_distancia
    angulo_barra_AE = np.arctan(d_AD / d_AB)
    Ax, Ay, Dx = calculo_reacciones(carga, angulo, d_AC)
    CE, BC = calculo_fuerzas_nodo_C(carga, angulo)
    BA, BE = calculo_fuerzas_nodo_B(BC)
    EA, ED = calculo_fuerzas_nodo_E(CE, angulo_barra_AE)
    DE, DA = calculo_fuerzas_nodo_D(Dx)
    AE, AB = calculo_fuerzas_nodo_A(Ax, Ay, angulo_barra_AE)
    fuerzas_internas = [("AB", BA, carga), ("AE", EA, carga), ("AD", DA, carga), ("BC", BC, carga), ("BE", BE, carga), ("CE", CE, carga), ("ED", ED, carga)]
    print(f"Aumento de distancia de:", d_AB - 2, "Las fuerzas calculadas son:")
    print(fuerzas_internas)
    print()

In [ ]:
# PARTE 4
variaciones_distancia = np.arange(2, 2.02 + 19 * 0.02, 0.02)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,20)
plt.style.use('seaborn-whitegrid')

fig , axes = plt.subplots(nrows=10,ncols=2)

d_AD = 1
d_BC = 2
for d_AB, ax in zip(variaciones_distancia, np.array(axes).flatten()):
    d_AC = d_AB + d_BC
    coordenada_x = np.array([0, d_AB, d_AC, d_AB, 0, 0, d_AB, d_AB])
    coordenada_Y = np.array([d_AD, d_AD, d_AD, 0, 0, d_AD, 0, d_AB])
    ax.plot(coordenada_x, coordenada_Y)
    ax.set_xticks([d_AB])
    ax.set_xticks([0, 1, d_AB, 3, 4])
    ax.set_title(f'Coordenadas cuando el aumento de posición es de {round(d_AB = 2, 3)}')
ax.set_xlabel('Posición del nodo B en x [0]')
plt.tight_layout()
plt.show

In [ ]:
# PARTE 5
variaciones_distancia = np.arange(2.02, 2.02 + 19 * 0.02, 0.02)
fuerzas_elemento_AB = np.array([])
fuerzas_elemento_AE = np.array([])
fuerzas_elemento_AD = np.array([])
fuerzas_elemento_BC = np.array([])
fuerzas_elemento_BE = np.array([])
fuerzas_elemento_CE = np.array([])
fuerzas_elemento_ED = np.array([])
counter = 0

for d_AB in variaciones_distancia:
    carga = 21.25
    angulo = 50
    d_AC = d_AB + d_BC
    angulo_barra_AE = np.arctan(d_AD / d_AB)
    Ax, Ay, Dx = calcular_reacciones(carga, angulo, d_AC)
    CE, BC = calculo_fuerzas_nodo_C(carga, angulo)
    BA, BE = calculo_fuerzas_nodo_B(BC)
    EA, ED = calculo_fuerzas_nodo_E(CE, angulo_barra_AE)
    DE, DA = calculo_fuerzas_nodo_D(Dx)
    AE, AB = calculo_fuerzas_nodo_A(Ax, Ay, angulo_barra_AE)
    fuerzas_elemento_AB = np.append([fuerzas_elemento_AB, BA])
    fuerzas_elemento_AE = np.append([fuerzas_elemento_AE, EA])
    fuerzas_elemento_AD = np.append([fuerzas_elemento_AD, AD])
    fuerzas_elemento_BC = np.append([fuerzas_elemento_BC, BC])
    fuerzas_elemento_BE = np.append([fuerzas_elemento_BE, BE])
    fuerzas_elemento_CE = np.append([fuerzas_elemento_CE, CE])
    fuerzas_elemento_ED = np.append([fuerzas_elemento_ED, ED])
    if counter == 5:
        print(f"Reacciones ngulo {angulo} grados:")
        print(Ax, Ay, Dx)
        print("Fuerzas internas:")    
        print(BA ,EA, DA ,BC ,BE ,CE ,ED)
    counter += 1

In [2]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,20)
plt.style.use('seaborn-whitegrid')
fuerzas_elemento = [(fuerzas_elemento_AB, "AB"), (fuerzas_elemento_AB, "AB"), (fuerzas_elemento_AE, "AE"), (fuerzas_elemento_AD, "AB"), (fuerzas_elemento_AD, "AD"), (fuerzas_elemento_BC, "BC"), (fuerzas_elemento_BE, "BE"), (fuerzas_elemento_CE, "CE"), (fuerzas_elemento_ED, "ED")]
fig, axes = plt.subplots(nrows=7, ncols=1, sharex=True)

for fuerzas, ax in zip(fuerzas_elementos, axes):
    ax.plot(variaciones_distancia, fuerzas[0])
    ax.fill_between(variaciones_distancia, fuerzas[1], alpha=0.25)
    ax.set.title(f'Esfuerzo del elemento {fuerza[1]} [Gpa] vs Posición del nodo B [m]')
ax.srt_xlabel('Posición del nodo B en x [m]')
plt.tight_layout()
plt.show

print(rango_de_angulos, rango_de_cargas)

NameError: name 'fuerzas_elemento_AB' is not defined